In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import optim

In [2]:
import sys
sys.path.insert(1, "../../")

In [3]:
from train import *
from data_preprocessing import *
from data_augmentation import *
from Models.darknet19 import *
import torch.optim.lr_scheduler as lr_scheduler

In [4]:
if torch.cuda.is_available():
    device=torch.device('cuda')
else:
    device=torch.device('cpu') 

In [5]:
model = Darknet19(num_classes=10, device=device, dtype=torch.float32)

In [6]:
data_path = '../../../datasets/imagenette2/'
norms_path = os.path.join(data_path, 'norms.json')

In [7]:
means = get_means(path=norms_path, train_loader=None)
stds = get_stds(path=norms_path, train_loader=None)

Means are: [0.44969913363456726, 0.44868946075439453, 0.45163223147392273]
stds are: [0.28648287057876587, 0.28796446323394775, 0.2865694761276245]


In [8]:
transformations = transforms.Compose([transforms.RandomResizedCrop((224, 224)),
                                              Augmentation(),
                                              transforms.ToTensor(),
                                              transforms.Normalize(mean=means, std=stds)])
transformations_val = transforms.Compose([transforms.Resize((224, 224)),
                                                 transforms.ToTensor(),
                                                 transforms.Normalize(mean=means, std=stds)
                                                 ])

In [9]:
train_dataset = ImageNetSubset(path=data_path, train=True, transform=transformations, half=False, show=False)
val_dataset = ImageNetSubset(path=data_path, train=False, transform=transformations_val, half=False, show=False)

In [10]:
epochs=10
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)

In [11]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0001)

In [12]:
loss_fn = nn.NLLLoss()

In [13]:
scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

In [14]:
history, gradient_stats = train(epochs, train_loader, val_loader, model, optimizer, loss_fn, scheduler, outputs_path='../log/darknet19/training/')

2024-12-08 17:46:05.408478 Epoch 1 
2024-12-08 17:46:19.843921 Batch 15 
2024-12-08 17:46:22.863686 Batch 30 
2024-12-08 17:46:25.832661 Batch 45 
2024-12-08 17:46:28.835159 Batch 60 
2024-12-08 17:46:31.819038 Batch 75 
2024-12-08 17:46:34.809809 Batch 90 
2024-12-08 17:46:37.786949 Batch 105 
2024-12-08 17:46:40.781802 Batch 120 
2024-12-08 17:46:43.841550 Batch 135 
[Train] Accuracy: 28.3134%, Loss per batch: 2.0235
2024-12-08 17:47:00.778264 Batch 15 
2024-12-08 17:47:02.594127 Batch 30 
2024-12-08 17:47:04.702893 Batch 45 
2024-12-08 17:47:06.643338 Batch 60 
[Val] Accuracy: 29.5541%, loss per batch: 2.3512
Epoch 1: SGD lr 0.0100 -> 0.0090
2024-12-08 17:47:07.797083 Epoch 2 
2024-12-08 17:47:22.502999 Batch 15 
2024-12-08 17:47:25.440560 Batch 30 
2024-12-08 17:47:28.443208 Batch 45 
2024-12-08 17:47:31.474548 Batch 60 
2024-12-08 17:47:34.710216 Batch 75 
2024-12-08 17:47:38.205006 Batch 90 
2024-12-08 17:47:41.580416 Batch 105 
2024-12-08 17:47:45.481623 Batch 120 
2024-12-08 17

In [15]:
history, gradient_stats = train(epochs, train_loader, val_loader, model, optimizer, loss_fn, scheduler, outputs_path='../log/darknet19/training/', resume=True)

C:\Me\PJAIT\Thesis\Code\yolov2-to-yolov8\Backbone experiments\Darknet19\../..\train.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(os.path.join(outp

2024-12-08 17:58:50.076927 Epoch 11 
2024-12-08 17:59:06.395768 Batch 15 
2024-12-08 17:59:11.176429 Batch 30 
2024-12-08 17:59:15.809725 Batch 45 
2024-12-08 17:59:20.404581 Batch 60 
2024-12-08 17:59:24.986545 Batch 75 
2024-12-08 17:59:29.620367 Batch 90 
2024-12-08 17:59:34.252686 Batch 105 
2024-12-08 17:59:38.904536 Batch 120 
2024-12-08 17:59:43.584828 Batch 135 
[Train] Accuracy: 62.5937%, Loss per batch: 1.1256
2024-12-08 18:00:02.256586 Batch 15 
2024-12-08 18:00:04.070312 Batch 30 
2024-12-08 18:00:06.376339 Batch 45 
2024-12-08 18:00:08.294606 Batch 60 
[Val] Accuracy: 64.3822%, loss per batch: 1.0676
Epoch 11: SGD lr 0.0035 -> 0.0031
2024-12-08 18:00:09.420463 Epoch 12 
2024-12-08 18:00:25.636371 Batch 15 
2024-12-08 18:00:30.172394 Batch 30 
2024-12-08 18:00:34.738872 Batch 45 
2024-12-08 18:00:39.236784 Batch 60 
2024-12-08 18:00:43.808451 Batch 75 
2024-12-08 18:00:48.407398 Batch 90 
2024-12-08 18:00:53.029752 Batch 105 
2024-12-08 18:00:57.683587 Batch 120 
2024-12-08